# JOSE LUIS VERDE SANCHEZ
19130002

In [71]:
import findspark
findspark.init()
findspark.find()
import pyspark
#Importamos sql
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [3]:
#Creamos cluster local
#sc = pyspark.SparkContext(appName="EstacionesCOAH")
#print(type(sc))
#sc.appName
#NO SE UTILIZA, HARE TODO CON PYSPARK SQL

In [6]:
#Creacion de la session
spark = SparkSession.builder.appName("EstacionesCOAH").getOrCreate()

In [7]:
#Base de la ruta para cada lugar
ruta = "C:\\Users\\verde\\Documents\\AGO2022\\BIGDATA\\JUPYTER\\CLIMA\\Estacion_"
rutasc = '__90_dias.csv'

In [8]:
#Genero la ruta de cada archivo
CANDELA = ruta+'CANDELA'+rutasc                 #0
CUATROCIENEGAS=ruta+'CUATROCIENEGAS'+rutasc     #1
MONCLOVA_ESIME=ruta+'MONCLOVA_ESIME'+rutasc     #2
MORELOSMUZQUIZ=ruta+'MORELOSMUZQUIZ'+rutasc     #3
NUEVAROSITA=ruta+'NUEVAROSITA'+rutasc           #4
OCAMPO=ruta+'OCAMPO'+rutasc                     #5
OCAMPOSMN=ruta+'OCAMPOSMN'+rutasc               #6
PIEDRAS_NEGRAS=ruta+'PIEDRAS_NEGRAS'+rutasc     #7
SALTILLO=ruta+'SALTILLO'+rutasc                 #8
SANTACECILIA=ruta+'SANTACECILIA'+rutasc         #9
TORREON=ruta+'TORREON'+rutasc                   #10
VENUSTIANOCARRANZASMN=ruta+'VENUSTIANOCARRANZASMN'+rutasc

In [47]:
#Hago una lista para poder iterarlo mas facilmente
lugares = [CANDELA,CUATROCIENEGAS,MONCLOVA_ESIME,MORELOSMUZQUIZ,NUEVAROSITA,OCAMPO,OCAMPOSMN,PIEDRAS_NEGRAS,
          SALTILLO,SANTACECILIA,TORREON,VENUSTIANOCARRANZASMN]
lugarestxt = ['CANDELA','CUATROCIENEGAS','MONCLOVA_ESIME','MORELOSMUZQUIZ','NUEVAROSITA','OCAMPO','OCAMPOSMN','PIEDRAS_NEGRAS',
          'SALTILLO','SANTACECILIA','TORREON','VENUSTIANOCARRANZASMN']

In [10]:
#Creo los dataframes
i=0
df=['','','','','','','','','','','','','']
for lugar in lugares:
    df[i] = spark.read.format("csv").option("inferSchema", False).option("header", True).option("sep",",").load(lugares[i])
    i=i+1

In [11]:
#Verifico informacion 0=CANDELA,etc
df[2].show()
#Como vemos se lee mal el csv dado que el "header" solamente tiene servicio meteorologico.

+--------------------------------+
|`Servicio Meteorol�gico Nacional|
+--------------------------------+
|                  `Red: SMN-ESMA|
|            `Estaci�n: MONCLO...|
|            `Estado: Coahuila...|
|            `Municipio: Monclova|
|            `Latitud (N): 26....|
|            `Longitud (O): -1...|
|                   `Altitud: 618|
|                               `|
|                     Fecha Local|
|             2022-08-31 20:30:00|
|             2022-08-31 20:20:00|
|             2022-08-31 20:10:00|
|             2022-08-31 20:00:00|
|             2022-08-31 19:50:00|
|             2022-08-31 19:40:00|
|             2022-08-31 19:30:00|
|             2022-08-31 19:20:00|
|             2022-08-31 19:10:00|
|             2022-08-31 19:00:00|
|             2022-08-31 18:50:00|
+--------------------------------+
only showing top 20 rows



In [12]:
#Como todas inician a mostrar los datos en la fila 10, en lugar de borrar datos de identificacion del CSV 
#podemos brincar a esa en especifico y mantener el csv aun default.
i=0
for lugar in lugares:
    df[i] = spark.read.format("csv").option("inferSchema", False).option("header", True).option("sep",",").\
    option("comment",'`').load(lugares[i])
    i=i+1

In [13]:
#Se manda llamar el df [3] = MORELOS
df[3].show()
#Se observa headers mal escritos y una columna _c11 generada porque los csv estan mal estructurados

+-------------------+-------------------+-----------------------------+----------------------------+------------------------+------------------------+-------------------------+--------------------+-------------------------+------------------+----------------------+----+
|        Fecha Local|          Fecha UTC|Direcci�n del Viento (grados)|Direcci�n de r�faga (grados)|Rapidez de viento (km/h)|Rapidez de r�faga (km/h)|Temperatura del Aire (�C)|Humedad relativa (%)|Presi�n Atmosf�rica (hpa)|Precipitaci�n (mm)|Radiaci�n Solar (W/m�)|_c11|
+-------------------+-------------------+-----------------------------+----------------------------+------------------------+------------------------+-------------------------+--------------------+-------------------------+------------------+----------------------+----+
|2022-11-15 09:10:00|2022-11-15 15:10:00|                         75.0|                        87.0|                     6.1|                     9.7|                      9.9|           

In [177]:
#Vemos cada header.
df[0].printSchema()
df2=['','','','','','','','','','','','','']
#Confirmo que _c11 es un extra que no contiene ningun dato.

root
 |-- Fecha Local: string (nullable = true)
 |-- Fecha UTC: string (nullable = true)
 |-- Temperatura del Aire (�C): string (nullable = true)
 |-- Precipitaci�n (mm): string (nullable = true)
 |-- Humedad relativa (%): string (nullable = true)
 |-- Presi�n Atmosf�rica (hpa): string (nullable = true)
 |-- Radiaci�n Solar (W/m�): string (nullable = true)
 |-- Direcci�n del Viento (grados): string (nullable = true)
 |-- Rapidez de viento (km/h): string (nullable = true)
 |-- Direcci�n de r�faga (grados): string (nullable = true)
 |-- Rapidez de r�faga (km/h): string (nullable = true)
 |-- _c11: string (nullable = true)



In [178]:
#Correccion a los header names
for i in range(12):
    df2[i] = df[i].withColumnRenamed("Temperatura del Aire (�C)","Temperatura del Aire")\
            .withColumnRenamed("Precipitaci�n (mm)","Precipitacion")\
            .withColumnRenamed("Presi�n Atmosf�rica (hpa)","Presion Atmosferica")\
            .withColumnRenamed("Radiaci�n Solar (W/m�)","Radiacion Solar")\
            .withColumnRenamed("Direcci�n del Viento (grados)","Direccion del Viento")\
            .withColumnRenamed("Direcci�n de r�faga (grados)","Direccion de rafaga")\
            .withColumnRenamed("Rapidez de r�faga (km/h)","Rapidez de rafaga")

In [179]:
#Verificacion de lo anterior
df2[11].printSchema()

root
 |-- Fecha Local: string (nullable = true)
 |-- Fecha UTC: string (nullable = true)
 |-- Direccion del Viento: string (nullable = true)
 |-- Direccion de rafaga: string (nullable = true)
 |-- Rapidez de viento (km/h): string (nullable = true)
 |-- Rapidez de rafaga: string (nullable = true)
 |-- Temperatura del Aire: string (nullable = true)
 |-- Humedad relativa (%): string (nullable = true)
 |-- Presion Atmosferica: string (nullable = true)
 |-- Precipitacion: string (nullable = true)
 |-- Radiacion Solar: string (nullable = true)
 |-- _c11: string (nullable = true)



In [180]:
#Eliminamos la extra con el comando drop y guardandolo en el mismo dataframe
for i in range(12):
    df2[i] = df2[i].drop(df2[i]._c11)    

In [181]:
#Comprobamos denuevo
df2[1].printSchema()

root
 |-- Fecha Local: string (nullable = true)
 |-- Fecha UTC: string (nullable = true)
 |-- Direccion del Viento: string (nullable = true)
 |-- Direccion de rafaga: string (nullable = true)
 |-- Rapidez de viento (km/h): string (nullable = true)
 |-- Rapidez de rafaga: string (nullable = true)
 |-- Temperatura del Aire: string (nullable = true)
 |-- Humedad relativa (%): string (nullable = true)
 |-- Presion Atmosferica: string (nullable = true)
 |-- Precipitacion: string (nullable = true)
 |-- Radiacion Solar: string (nullable = true)



In [36]:
#Ahora vemos como va quedando el formato.
#Ya no tiene datos null y todos los header pueden leerse.
df2[1].show(3)

+-------------------+-------------------+--------------------+-------------------+------------------------+-----------------+--------------------+--------------------+-------------------+-------------+---------------+
|        Fecha Local|          Fecha UTC|Direccion del Viento|Direccion de rafaga|Rapidez de viento (km/h)|Rapidez de rafaga|Temperatura del Aire|Humedad relativa (%)|Presion Atmosferica|Precipitacion|Radiacion Solar|
+-------------------+-------------------+--------------------+-------------------+------------------------+-----------------+--------------------+--------------------+-------------------+-------------+---------------+
|2022-11-15 09:10:00|2022-11-15 15:10:00|               110.0|              133.0|                    24.7|             41.7|                12.8|                  32|                938|            0|            377|
|2022-11-15 09:00:00|2022-11-15 15:00:00|               103.0|              126.0|                    22.3|             40.5|   

In [182]:
#Como tenemos doble dato de fecha solo en distinto zona borramos uno para tener aun mas limpio el DF
for i in range(12):
    df2[i] = df2[i].drop("Fecha Local")    

In [183]:
#Mostramos para ver cambios.
df2[1].show(3)

+-------------------+--------------------+-------------------+------------------------+-----------------+--------------------+--------------------+-------------------+-------------+---------------+
|          Fecha UTC|Direccion del Viento|Direccion de rafaga|Rapidez de viento (km/h)|Rapidez de rafaga|Temperatura del Aire|Humedad relativa (%)|Presion Atmosferica|Precipitacion|Radiacion Solar|
+-------------------+--------------------+-------------------+------------------------+-----------------+--------------------+--------------------+-------------------+-------------+---------------+
|2022-11-15 15:10:00|               110.0|              133.0|                    24.7|             41.7|                12.8|                  32|                938|            0|            377|
|2022-11-15 15:00:00|               103.0|              126.0|                    22.3|             40.5|                12.4|                  32|              937.9|            0|            332|
|2022-11-1

In [43]:
#Ahora que tenemos el DF usable podemos comenzar, primero veremos de cuantos datos estamos hablando.
datos=0
for i in range(12):
    datos=datos+df[i].count()
datos

82557

In [73]:
#Como primer ejemplo vamos a ver la diferencia de lluvia en los lugares.
#Con el filter se eliminan los null de la tabla.
for i in range(12):
    print(lugarestxt[i])
    df2[i].groupBy(df2[i]['Precipitacion']).count().filter("`Precipitacion` != ''").sort(desc('count')).show()

CANDELA
+-------------+-----+
|Precipitacion|count|
+-------------+-----+
+-------------+-----+

CUATROCIENEGAS
+-------------+-----+
|Precipitacion|count|
+-------------+-----+
|            0|12857|
|          0.2|   23|
|          0.4|   16|
|          0.6|    6|
|          0.8|    6|
|          1.2|    3|
|            1|    3|
|          1.4|    2|
|          2.6|    2|
|          1.6|    2|
|          2.2|    2|
|          3.4|    1|
|            4|    1|
|          8.8|    1|
|            2|    1|
|          3.2|    1|
|         15.2|    1|
+-------------+-----+

MONCLOVA_ESIME
+-------------+-----+
|Precipitacion|count|
+-------------+-----+
|            0|   72|
|         0.51|    3|
|         1.02|    2|
|         3.05|    1|
|         2.29|    1|
|         1.52|    1|
|         4.32|    1|
+-------------+-----+

MORELOSMUZQUIZ
+-------------+-----+
|Precipitacion|count|
+-------------+-----+
|            0|12395|
|          0.2|  113|
|          0.4|   62|
|          0.8|   30

In [124]:
#Para hacer algunas cosas usaremos solamente saltillo [8]. //TORREON NO TENIA DATOS//
#Se usara un solo dataset para hacer mas facil leer los resultados...
cols=['Fecha UTC', 'Temperatura del Aire','Presion Atmosferica', 'Precipitacion']
saltillo= df2[8].select(*cols)
saltillo.show()

+-------------------+--------------------+-------------------+-------------+
|          Fecha UTC|Temperatura del Aire|Presion Atmosferica|Precipitacion|
+-------------------+--------------------+-------------------+-------------+
|2022-11-15 15:10:00|                11.9|              826.7|            0|
|2022-11-15 15:00:00|                11.9|              826.7|            0|
|2022-11-15 14:50:00|                11.6|              826.7|            0|
|2022-11-15 14:40:00|                10.8|              826.7|            0|
|2022-11-15 14:30:00|                10.8|              826.5|            0|
|2022-11-15 14:20:00|                10.7|              826.4|            0|
|2022-11-15 14:10:00|                 9.9|              826.4|            0|
|2022-11-15 14:00:00|                 8.9|              826.4|            0|
|2022-11-15 13:50:00|                 7.7|              826.2|            0|
|2022-11-15 13:40:00|                 6.1|                826|            0|

In [80]:
from pyspark.sql.types import *

In [125]:
#Ahora por ejemplo si queremos sacar temp max tendriamos que transformar la columna de temp a float
saltillo = saltillo.withColumn("Temperatura del Aire",saltillo["Temperatura del Aire"].cast("float"))

In [84]:
#Comprobamos que se cambio correctamente
saltillo.dtypes

[('Fecha UTC', 'string'),
 ('Temperatura del Aire', 'float'),
 ('Presion Atmosferica', 'string'),
 ('Precipitacion', 'string')]

In [100]:
tmax=saltillo.approxQuantile("Temperatura del Aire",[0.9999999],0.000001) #Con la funcion quantile

In [101]:
str(tmax)

'[32.20000076293945]'

In [110]:
#Ahora con la funcion .max()
saltillo.groupBy("Precipitacion").max("Temperatura del Aire").sort(desc("max(Temperatura del Aire)")).show()
#En esta funcion por el groupby podemos hacer mas cosas que con quantile, aqui podemos ver la temp maxima por cada
#Diferente lluvia, donde nos damos cuenta que entre mas fuerte este lloviendo mas baja es la temperatura.
#Aparte se confirmo que la temp mas caliente si fue 32.2 grados centigrados

+-------------+-------------------------+
|Precipitacion|max(Temperatura del Aire)|
+-------------+-------------------------+
|            0|                     32.2|
|          0.2|                     26.9|
|          0.8|                     23.7|
|          0.6|                     22.1|
|          2.2|                     21.4|
|            1|                     21.1|
|          1.8|                     20.2|
|          0.4|                     19.9|
|          1.6|                     19.7|
|          1.2|                     19.4|
|          3.2|                     18.8|
|          5.6|                     18.5|
|          2.4|                     18.1|
|          1.4|                     17.3|
|          3.6|                     17.0|
|            3|                     16.8|
|            2|                     16.8|
|          2.6|                     16.8|
|          3.4|                     16.4|
|          2.8|                     16.0|
+-------------+-------------------

In [118]:
saltillo.describe('Temperatura del Aire').show()
#Con describe nos ahorramos el llamar independientemente cada metodo como anteriormente
#Aqui se ve el promedio de temp, 18, el minimo y el max, mas el numero de datos que se leyeron.

+-------+--------------------+
|summary|Temperatura del Aire|
+-------+--------------------+
|  count|               12945|
|   mean|   18.05146389119917|
| stddev|   5.637346465700087|
|    min|                 2.1|
|    max|                32.2|
+-------+--------------------+



In [135]:
#Se genera un temporal con la pura fecha para poder hacer pruebas por dias.
temp=saltillo.withColumn("FECHA",to_date("Fecha UTC"))
temp.show()

+-------------------+--------------------+-------------------+-------------+----------+
|          Fecha UTC|Temperatura del Aire|Presion Atmosferica|Precipitacion|     FECHA|
+-------------------+--------------------+-------------------+-------------+----------+
|2022-11-15 15:10:00|                11.9|              826.7|            0|2022-11-15|
|2022-11-15 15:00:00|                11.9|              826.7|            0|2022-11-15|
|2022-11-15 14:50:00|                11.6|              826.7|            0|2022-11-15|
|2022-11-15 14:40:00|                10.8|              826.7|            0|2022-11-15|
|2022-11-15 14:30:00|                10.8|              826.5|            0|2022-11-15|
|2022-11-15 14:20:00|                10.7|              826.4|            0|2022-11-15|
|2022-11-15 14:10:00|                 9.9|              826.4|            0|2022-11-15|
|2022-11-15 14:00:00|                 8.9|              826.4|            0|2022-11-15|
|2022-11-15 13:50:00|           

In [142]:
#Ahora podemos ver cuantos registros se hicieron diarios.
regdia=temp.groupBy("FECHA").agg(count("Fecha UTC")).sort(desc("FECHA"))
regdia.show(20) #Mostramos los primeros 20
regdia.count()#DIAS REGISTRADOS

+----------+----------------+
|     FECHA|count(Fecha UTC)|
+----------+----------------+
|2022-11-15|              92|
|2022-11-14|             144|
|2022-11-13|             144|
|2022-11-12|             144|
|2022-11-11|             144|
|2022-11-10|             144|
|2022-11-09|             144|
|2022-11-08|             144|
|2022-11-07|             144|
|2022-11-06|             144|
|2022-11-05|             144|
|2022-11-04|             144|
|2022-11-03|             144|
|2022-11-02|             144|
|2022-11-01|             144|
|2022-10-31|             144|
|2022-10-30|             138|
|2022-10-29|             144|
|2022-10-28|             144|
|2022-10-27|             144|
+----------+----------------+
only showing top 20 rows



91

In [144]:
#Teniendo los temporales diarios podemos agrupar para ver maximos y minimos o promedios por dia.
#Tambien hacer busquedas por dia en especial.
diario=temp.groupBy("FECHA")


In [152]:
#El promedio diario de la temperatura del aire y maximo
diario.mean().sort(desc("avg(Temperatura del Aire)")).show()
diario.max().sort(desc("max(Temperatura del Aire)")).show()

+----------+-------------------------+
|     FECHA|avg(Temperatura del Aire)|
+----------+-------------------------+
|2022-08-17|        28.15370379553901|
|2022-08-21|        23.90347222487132|
|2022-08-22|       23.341666724946762|
|2022-08-18|       23.270138806766933|
|2022-08-20|       22.964583317438763|
|2022-08-19|       22.684892064375845|
|2022-08-28|       22.229861074023777|
|2022-08-29|       22.114583333333332|
|2022-08-23|       21.984057979307313|
|2022-08-24|        21.83958339691162|
|2022-08-25|          21.727083351877|
|2022-10-24|        21.52013895246718|
|2022-11-04|       21.332638919353485|
|2022-08-30|         21.1513888835907|
|2022-08-31|       20.918055560853745|
|2022-10-16|        20.85972218381034|
|2022-09-01|       20.624305579397415|
|2022-09-09|       20.540972265932297|
|2022-11-03|        20.45694437291887|
|2022-09-10|        20.45555555820465|
+----------+-------------------------+
only showing top 20 rows

+----------+-------------------------+

In [154]:
#Cambiamos los type de las demas columnas para poder hacer pruebas con ellas tambien
temp = temp.withColumn("Presion Atmosferica",temp["Presion Atmosferica"].cast("float"))
temp = temp.withColumn("Precipitacion",temp["Precipitacion"].cast("float"))

In [155]:
#Comprobamos que se cambio correctamente el type
temp.dtypes

[('Fecha UTC', 'string'),
 ('Temperatura del Aire', 'float'),
 ('Presion Atmosferica', 'float'),
 ('Precipitacion', 'float'),
 ('FECHA', 'date')]

In [156]:
#Generamos el groupby dario con los nuevos datos.
diario=temp.groupBy("FECHA")

In [163]:
#Mostramos el dataframe con cada maximo por dia
diario.max().sort(desc("FECHA")).show(91)

+----------+-------------------------+------------------------+------------------+
|     FECHA|max(Temperatura del Aire)|max(Presion Atmosferica)|max(Precipitacion)|
+----------+-------------------------+------------------------+------------------+
|2022-11-15|                     17.3|                   826.7|               0.0|
|2022-11-14|                     27.4|                   822.0|               0.0|
|2022-11-13|                     21.9|                   825.2|               0.0|
|2022-11-12|                     18.5|                   826.2|               0.0|
|2022-11-11|                     27.3|                   820.7|               0.0|
|2022-11-10|                     29.7|                   824.3|               0.0|
|2022-11-09|                     27.9|                   827.4|               0.0|
|2022-11-08|                     26.0|                   828.0|               0.0|
|2022-11-07|                     27.1|                   826.6|               0.0|
|202

In [194]:
#Para hacer uso de todos los datasets se hara lo mismo que se hizo con Saltillo en todos al mismo tiempo
df3=['','','','','','','','','','','','','']   
#----
#En caso de no querer un array de dataframes
#CANDELA = df2[0].select(*cols)                 #0
#CUATROCIENEGAS=df2[1].select(*cols)     #1
#MONCLOVA_ESIME=df2[2].select(*cols)     #2
#MORELOSMUZQUIZ=df2[3].select(*cols)    #3
#NUEVAROSITA=df2[4].select(*cols)           #4
#OCAMPO=df2[5].select(*cols)                    #5
#OCAMPOSMN=df2[6].select(*cols)              #6
#PIEDRAS_NEGRAS=df2[7].select(*cols)     #7
#SALTILLO=df2[8].select(*cols)                 #8
#SANTACECILIA=df2[9].select(*cols)        #9
#TORREON=df2[10].select(*cols)                 #10
#VENUSTIANOCARRANZASMN=df2[11].select(*cols)   #11
#----
cols=['Fecha UTC', 'Temperatura del Aire','Presion Atmosferica', 'Precipitacion']
for i in range(12):
    df3[i]= df2[i].select(*cols)


In [195]:
#Convertimos los valores de string a float de cada dataframe
for i in range(12):
    df3[i]=df3[i].withColumn("Temperatura del Aire",df3[i]["Temperatura del Aire"].cast("float"))\
    .withColumn("Precipitacion",df3[i]["Precipitacion"].cast("float"))\
    .withColumn("Presion Atmosferica",df3[i]["Presion Atmosferica"].cast("float"))

In [198]:
#Mostramos las tablas resultantes
for i in range(12):
    print(lugarestxt[i])
    df3[i].show(5)

CANDELA
+---------+--------------------+-------------------+-------------+
|Fecha UTC|Temperatura del Aire|Presion Atmosferica|Precipitacion|
+---------+--------------------+-------------------+-------------+
+---------+--------------------+-------------------+-------------+

CUATROCIENEGAS
+-------------------+--------------------+-------------------+-------------+
|          Fecha UTC|Temperatura del Aire|Presion Atmosferica|Precipitacion|
+-------------------+--------------------+-------------------+-------------+
|2022-11-15 15:10:00|                12.8|              938.0|          0.0|
|2022-11-15 15:00:00|                12.4|              937.9|          0.0|
|2022-11-15 14:50:00|                12.2|              937.9|          0.0|
|2022-11-15 14:40:00|                11.9|              937.9|          0.0|
|2022-11-15 14:30:00|                11.6|              937.9|          0.0|
+-------------------+--------------------+-------------------+-------------+
only showing to

In [200]:
#Mostramos los datos importantes de cada dataframe sobre la temperatura del aire
for i in range(12):
    print(lugarestxt[i])
    df3[i].describe('Temperatura del Aire').show()

CANDELA
+-------+--------------------+
|summary|Temperatura del Aire|
+-------+--------------------+
|  count|                   0|
|   mean|                null|
| stddev|                null|
|    min|                null|
|    max|                null|
+-------+--------------------+

CUATROCIENEGAS
+-------+--------------------+
|summary|Temperatura del Aire|
+-------+--------------------+
|  count|               12928|
|   mean|   23.72945543918943|
| stddev|   5.823517549273008|
|    min|                 3.5|
|    max|                36.6|
+-------+--------------------+

MONCLOVA_ESIME
+-------+--------------------+
|summary|Temperatura del Aire|
+-------+--------------------+
|  count|                 487|
|   mean|  26.142094398670864|
| stddev|   3.406133133183297|
|    min|                22.0|
|    max|                34.8|
+-------+--------------------+

MORELOSMUZQUIZ
+-------+--------------------+
|summary|Temperatura del Aire|
+-------+--------------------+
|  count|     

In [201]:
#Hacemos override al df3 para agregarale la columna de fecha.
for i in range(12):
    df3[i]=df3[i].withColumn("FECHA",to_date("Fecha UTC"))

In [209]:
rd=['','','','','','','','','','','','',''] 
#Mostramos cantidad de registros diarios de cada lugar
for i in range(12):
    print(lugarestxt[i])#Mostrar nombre del lugar
    rd[i]=df3[i].groupBy("FECHA").agg(count("Fecha UTC")).sort(desc("FECHA")) #AGRUPAR TODOS LOS MISMOS DIAS
    rd[i].show(5) #Mostramos los primeros 20
    print("Dias registrados: "+str(rd[i].count()))  #DIAS REGISTRADOS
    print("+-----------------------+\n")

CANDELA
+-----+----------------+
|FECHA|count(Fecha UTC)|
+-----+----------------+
+-----+----------------+

Dias registrados: 0
+-----------------------+

CUATROCIENEGAS
+----------+----------------+
|     FECHA|count(Fecha UTC)|
+----------+----------------+
|2022-11-15|              92|
|2022-11-14|             144|
|2022-11-13|             144|
|2022-11-12|             144|
|2022-11-11|             144|
+----------+----------------+
only showing top 5 rows

Dias registrados: 91
+-----------------------+

MONCLOVA_ESIME
+----------+----------------+
|     FECHA|count(Fecha UTC)|
+----------+----------------+
|2022-09-01|              10|
|2022-08-31|             144|
|2022-08-30|              46|
|2022-08-25|              18|
|2022-08-24|             144|
+----------+----------------+
only showing top 5 rows

Dias registrados: 6
+-----------------------+

MORELOSMUZQUIZ
+----------+----------------+
|     FECHA|count(Fecha UTC)|
+----------+----------------+
|2022-11-15|            

In [210]:
#Generamos el groupby dario con los nuevos datos de todos los dataframe
for i in range(12):
    rd[i]=df3[i].groupBy("FECHA")

In [212]:
#MOSTRAMOS EL MAXIMO DE CADA DIA DE TODOS LOS DIAS REGISTRADOS POR CADA DATAFRAME
for i in range(12):
    print(lugarestxt[i])
    rd[i].max().sort(desc("max(Temperatura del Aire)")).show(5) #TOP 5 TEMPERATURAS MAS CALIENTES DE CADA LUGAR
#Piedras negras 40.7 grados

CANDELA
+-----+-------------------------+------------------------+------------------+
|FECHA|max(Temperatura del Aire)|max(Presion Atmosferica)|max(Precipitacion)|
+-----+-------------------------+------------------------+------------------+
+-----+-------------------------+------------------------+------------------+

CUATROCIENEGAS
+----------+-------------------------+------------------------+------------------+
|     FECHA|max(Temperatura del Aire)|max(Presion Atmosferica)|max(Precipitacion)|
+----------+-------------------------+------------------------+------------------+
|2022-08-22|                     36.6|                   930.0|               0.2|
|2022-08-21|                     36.3|                   931.3|               0.0|
|2022-11-04|                     35.3|                   927.2|               0.0|
|2022-08-29|                     34.9|                   928.2|               0.0|
|2022-08-19|                     34.7|                   929.3|               0.0|


In [213]:
spark.stop()